# Drug Discovery Project

In [1]:
# Import
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem

In [3]:
path = Path('../dataset/13321_2017_226_MOESM1_ESM/')
#df = pd.read_csv('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl.csv', index_col=0)

In [4]:
#df.head()
list(path.iterdir())

[PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl_ecfp_1024.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL1978_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL218_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/.DS_Store'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/mol_images'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL240_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/RdkitDescriptors.py'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL219_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL301_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL244_cl.csv')]

# RdkitDescriptors

In [5]:
# function for returning fingerprint from a specific smile.

def fp(smile, diam = 2, bits = 1024):

    mol = Chem.MolFromSmiles(smile)
    Chem.SanitizeMol(mol)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, diam, nBits = bits) 
    return fp

In [6]:
#ECFP4
#Generated Circular fingerprints hashed into n bits length vectors.

def ECFP(ifile, ofile, diam, bits):
    
    print(f"Reading data from file: {ifile}")
    df = pd.read_csv(ifile)
    print(f"Headers: {list(df.columns)}")
    
    df.insert(2, "ECFP4_", df.SMILES.apply(fp))
    
    df[[f"ECFP4_{i+1}" for i in range(len(df.ECFP4_[0]))]] = df.ECFP4_.to_list()
    
    df.drop("ECFP4_", axis = 1, inplace = True)
    
    
    df.to_csv(DATA/ofile, index = None)
    return df

# Run the functions on a file from dataset and store the results

In [7]:
# ECFP(DATA/'CHEMBL205_cl.csv', './CHEMBL205_cl_ecfp_1024.csv', 2, 1024)

In [8]:
df = pd.read_csv(path/'CHEMBL205_cl_ecfp_1024.csv')

In [9]:
df.head()

,CID,SMILES,Activity,ECFP4_1,ECFP4_2,ECFP4_3,ECFP4_4,ECFP4_5,ECFP4_6,ECFP4_7,...,ECFP4_1015,ECFP4_1016,ECFP4_1017,ECFP4_1018,ECFP4_1019,ECFP4_1020,ECFP4_1021,ECFP4_1022,ECFP4_1023,ECFP4_1024
0,CHEMBL188002,S(=O)(=O)(N)c1cc(N/C(/S)=N\c2cc(C(=O)[O-])c(cc...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CHEMBL364127,Clc1ccc(cc1)C(=O)NC1Cc2cc(S(=O)(=O)N)ccc2C1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,CHEMBL1683469,S(=O)(=O)(N)c1ccc(cc1)CNS(=O)(=O)CC12CCC(CC1=O...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,CHEMBL52564,Oc1ccccc1\C=C\C(=O)[O-],1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CHEMBL21427,OB(O)c1ccc(OC)cc1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Split data

In [10]:
# Split into X,y
X, y = df.drop(["CID", "SMILES", "Activity"], axis=1), df["Activity"]

In [11]:
# check info of dataframe
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17941 entries, 0 to 17940
Columns: 1024 entries, ECFP4_1 to ECFP4_1024
dtypes: int64(1024)
memory usage: 140.2 MB


In [12]:
# y is a pandas series
y.head(), y.size, type(y)

(0    1
 1    1
 2    1
 3    1
 4    1
 Name: Activity, dtype: int64,
 17941,
 pandas.core.series.Series)

# Train test split

In [13]:
from sklearn.model_selection import train_test_split, KFold

In [14]:
# regular train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

# 5-Fold Cross Validation

In [15]:
#5-fold
kf = KFold(n_splits=5, shuffle=True, random_state=999)

In [16]:
# append to a list / could write to csv file to keep integrity
X_train_list, X_valid_list, y_train_list, y_valid_list = [], [], [], []

for train_index, valid_index in kf.split(X_train):
    X_train_list.append(X_train.iloc[train_index])
    X_valid_list.append(X_train.iloc[valid_index])
    y_train_list.append(y_train.iloc[train_index])
    y_valid_list.append(y_train.iloc[valid_index])
    
   

In [17]:
y_train_list[0].head()

14960    0
16280    0
16068    0
13692    0
14043    0
Name: Activity, dtype: int64

In [18]:
# TODO: add splits to csv file

# Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
from sklearn.metrics import auc,roc_auc_score,recall_score,precision_score,f1_score
from  sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

In [21]:
# train method for Random Forest
def train_rf(X_train, X_test, y_train, y_test, n_estimators=5, criterion='gini', max_features='log2'):

    
    rf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, min_samples_split=2, max_features=max_features, 
                               max_leaf_nodes=None,bootstrap=False,oob_score=False, n_jobs=-1, random_state=69)
    
    rf.fit(X_train,y_train)
    y_pred= rf.predict(X_test)
    y_pred_prob=rf.predict_proba(X_test)
    
    temp=[]
    for j in range(len(y_pred_prob)):
        temp.append(y_pred_prob[j][1])
    auc=roc_auc_score(np.array(y_test),np.array(temp))
    acc2=accuracy_score(y_test,y_pred)
    mcc=matthews_corrcoef(y_test,y_pred)
    Recall=recall_score(y_test, y_pred,pos_label=1)
    Precision=precision_score(y_test, y_pred,pos_label=1)
    F1_score=f1_score(y_test, y_pred,pos_label=1)

    return auc,acc2,mcc,Recall,Precision,F1_score

In [22]:
# Train and print scores for each split
for i in range(0,5):
    
    print(f'Cross validation iteration: {i + 1}')
    
    X_train = X_train_list[i]
    X_valid = X_valid_list[i]
    y_train = y_train_list[i]
    y_valid = y_valid_list[i]
    
    auc,acc2,mcc,Recall,Precision,F1_score = train_rf(X_train, X_valid, y_train, y_valid, 
                                                      n_estimators=5, criterion='gini', max_features='log2')
    
    print(f'AUC score:{auc}')
    print(f'Accuracy: {acc2}')
    print(f'Matthews: {mcc}')
    print(f'Recall: {Recall}')
    print(f'Precision: {Precision}')
    print(f'F1 score: {F1_score}')
    print()

Cross validation iteration: 1
AUC score:0.9574881629628877
Accuracy: 0.9614491407338597
Matthews: 0.7644270608298096
Recall: 0.7475247524752475
Precision: 0.825136612021858
F1 score: 0.7844155844155845

Cross validation iteration: 2
AUC score:0.966627415949326
Accuracy: 0.967487227124942
Matthews: 0.7842964328759628
Recall: 0.7393617021276596
Precision: 0.86875
F1 score: 0.7988505747126436

Cross validation iteration: 3
AUC score:0.9695417602840419
Accuracy: 0.9679516953088714
Matthews: 0.7914678805284223
Recall: 0.7978142076502732
Precision: 0.8202247191011236
F1 score: 0.8088642659279779

Cross validation iteration: 4
AUC score:0.9251725594830793
Accuracy: 0.9595912679981421
Matthews: 0.7719589551000242
Recall: 0.7256637168141593
Precision: 0.8677248677248677
F1 score: 0.7903614457831326

Cross validation iteration: 5
AUC score:0.9679631282423161
Accuracy: 0.9721189591078067
Matthews: 0.8181825478837996
Recall: 0.8241758241758241
Precision: 0.8426966292134831
F1 score: 0.833333333333